# Setup

The main focus of this notebook is to analyze and mitigate potential biases in a dataset. The notebook uses various techniques and technologies, such as [AIF360](https://aif360.mybluemix.net), [Fairlearn](https://fairlearn.org) , and [scikit-learn](https://scikit-learn.org/stable/), to preprocess and analyze the data, and to train and evaluate machine learning models. The notebook also includes visualizations and statistics to help understand the distribution and correlations of the data, and to identify any potential biases.

**PLEASE NOTE**: The notebook must be configured with a dataset and some configuration variables.  
The ***protected attributes*** must be categorical and binary (0,1), but the original column must be mantained as a continuous variable.

## Colab
In order to run this notebook in Google Colab, you have to:
1. Upload the project folder to your Google Drive
2. Mount your Google Drive in the next code cell
3. Update the `path_to_project` variable, with the path to the project folder in your Google Drive (e.g. `path_to_project = '/content/drive/MyDrive/Colab/project'`), this can be seen from the file tab on the left of the Colab interface
4. Update the pip installation command  (e.g. `!pip install -r /content/drive/MyDrive/Colab/project/requirements.txt`)

In [1]:
try:
  from google.colab import drive
  drive.mount('/content/drive')
  import sys
  path_to_project = '/content/drive/MyDrive//FairAlgorithm'  #UPDATE THIS LINE
  sys.path.append(path_to_project)
  !sudo apt install libcairo2-dev pkg-config python3-dev
  %pip install -r /content/drive/MyDrive/FairAlgorithm/source/requirements.txt  #UPDATE THIS LINE
  IN_COLAB = True
except:
  IN_COLAB = False

Mounted at /content/drive
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
python3-dev is already the newest version (3.10.6-1~22.04.1).
python3-dev set to manually installed.
The following packages were automatically installed and are no longer required:
  libbz2-dev libpkgconf3 libreadline-dev
Use 'sudo apt autoremove' to remove them.
The following additional packages will be installed:
  libblkid-dev libblkid1 libcairo-script-interpreter2 libffi-dev
  libglib2.0-dev libglib2.0-dev-bin libice-dev liblzo2-2 libmount-dev
  libmount1 libpixman-1-dev libselinux1-dev libsepol-dev libsm-dev
  libxcb-render0-dev libxcb-shm0-dev
Suggested packages:
  libcairo2-doc libgirepository1.0-dev libglib2.0-doc libgdk-pixbuf2.0-bin
  | libgdk-pixbuf2.0-dev libxml2-utils libice-doc cryptsetup-bin libsm-doc
The following packages will be REMOVED:
  pkgconf r-base-dev
The following NEW packages will be installed:
  libblkid-dev libcairo-script-interpreter2 

In [2]:
import numpy as np
import pandas as pd
np.random.seed(1234)
from rich import print
from rich.columns import Columns
from rich.panel import Panel
from rich.align import Align
from source.utils.print_util import *
from source.utils.data_preprocessing import *
import matplotlib.pyplot as plt
from fairlearn.metrics import  MetricFrame, count, false_negative_rate, false_positive_rate, selection_rate, equalized_odds_difference, demographic_parity_difference
from fairlearn.preprocessing import CorrelationRemover
from fairlearn.adversarial import AdversarialFairnessClassifier
from fairlearn.reductions import ExponentiatedGradient, Moment
from fairlearn.postprocessing import ThresholdOptimizer
from aif360.datasets import BinaryLabelDataset, StructuredDataset, StandardDataset
from aif360.metrics import BinaryLabelDatasetMetric, ClassificationMetric, Metric
from aif360.algorithms.preprocessing import DisparateImpactRemover, Reweighing, LFR, OptimPreproc
from aif360.algorithms.preprocessing.optim_preproc_helpers.opt_tools import OptTools
from aif360.algorithms.inprocessing import PrejudiceRemover, AdversarialDebiasing, ExponentiatedGradientReduction
from aif360.algorithms.postprocessing import RejectOptionClassification, CalibratedEqOddsPostprocessing, EqOddsPostprocessing
from aif360.algorithms import Transformer
import tensorflow.compat.v1 as tf
tf.disable_eager_execution()

from tqdm.notebook import tqdm
import pickle

pip install 'aif360[inFairness]'


In [3]:
from sklearn.model_selection import cross_validate, cross_val_score, cross_val_predict, train_test_split, StratifiedKFold
from sklearn.metrics import classification_report, recall_score, accuracy_score, precision_score, confusion_matrix, roc_curve
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, AdaBoostClassifier, BaggingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, normalize


In [4]:
!pip install BlackBoxAuditing

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 32.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for BlackBoxAuditing: filename=BlackBoxAuditing-0.1.54-py2.py3-none-any.whl size=1394756 sha256=0e0a572dc04598b7227dbd6dcdc756e6a1fb88860718df4cfa21ee8c7f8e9be5
  Stored in directory: /root/.cache/pip/wheels/c0/4f/b1/80e1b0790df07536470758fe0a4f9ff8fa942fd9fe30bbb192
Successfully built BlackBoxAuditing


## Configure the notebook


In the next code cell, set all the variables that are used throughout the notebook.  
The variables are used to configure the notebook, and to set the paths to the data files.

Modify the variables in the next code cell to configure the notebook

- `dataset_name`: The name of the dataset file.
- `dataset_path`: The path to the dataset file.
- `target`: The target feature to predict.
- `target_variable_labels`: The labels for the target feature.
- `sensible_attribute`: The sensible attribute to use for bias mitigation.

In [5]:
#INPUT
dataset_name = "diabetes-prediction"

if dataset_name == "diabetes-women":
  ignore_cols = ['Age']
  target_variable = 'Outcome'
  target_variable_labels= ['No Diabetes','Diabetes']
  sensible_attribute = 'AgeCategory'

  default_mappings = {
      'label_maps': [{1.0: 'Diabetic', 0.0: 'NonDiabetic'}],
      'protected_attribute_maps': [{1.0: 'Adult', 0.0: 'Young'}]
  }

elif dataset_name == "diabetes-prediction":
  ignore_cols = []
  target_variable = 'diabetes'
  target_variable_labels= ['No Diabetes','Diabetes']
  sensible_attribute = 'race_category'

  default_mappings = {
      'label_maps': [{1.0: 'Diabetic', 0.0: 'NonDiabetic'}],
      'protected_attribute_maps': [{1.0: 'Caucasian', 0.0: 'Non-Caucasian'}]
  }

dataset_path = path_to_project + '/data/preprocessed/preprocessed-{}.csv'.format(dataset_name) if IN_COLAB else 'data/preprocessed/preprocessed-{}.csv'.format(dataset_name)
df = pd.read_csv(dataset_path)
df = df.drop(columns=ignore_cols)
feature_cols = df.columns

In [6]:
from sklearn import preprocessing

# Build a list with format (feature, correlation with target)
features_corr = [(column, correlation) for column, correlation in zip(df.columns, df.corr()[target_variable])] # Modify stroke with the target_variable

# Sort the features by correlation
sorted_features = sorted(features_corr, key=lambda x: x[1], reverse=True)

# Clean and take top 4
main_features = [feature[0] for feature in sorted_features if feature[1] > 0][:5]

# Add sensitive attribute
main_features = main_features + [sensible_attribute]

# Create a new reduced df
df_reduced = df[main_features]
df_reduced = df_reduced[:5000]

In [7]:
n_estimators = 30
random_seed = 1234
n_splits= 10
apply_dimensionality_reduction = True

models = {'Logistic Regression': LogisticRegression(solver='liblinear', max_iter=50), #sag
          'Decision Tree':DecisionTreeClassifier(max_depth=None),
          'Bagging':BaggingClassifier(DecisionTreeClassifier(max_depth=3),n_estimators=n_estimators),
          'Random Forest':RandomForestClassifier(n_estimators=n_estimators),
          'Extremely Randomized Trees':ExtraTreesClassifier(n_estimators=n_estimators),
          'Ada Boost':AdaBoostClassifier(DecisionTreeClassifier(max_depth=3),n_estimators=n_estimators)}

In [8]:
config = {}
config['df']= df
config['target_variable'] = target_variable
config['sensible_attribute'] = sensible_attribute
config['path_to_project'] = path_to_project
config['n_splits'] = n_splits
config['models'] = models
config['n_estimators'] = n_estimators
config['random_seed'] = random_seed

In [9]:
def unpack_config(config):
  return config['df'], config['target_variable'], config['sensible_attribute'], config['path_to_project'], config['n_splits'], config['models'], config['n_estimators'],config['random_seed']

## Load the data
Data is loaded from the file specified in the `dataset_path` variable using the `pandas` library.  
[pandas](https://pandas.pydata.org) is a fast, powerful, flexible and easy to use open source data analysis and manipulation tool,
built on top of the Python programming language.

In [10]:
df.head(5)

,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses,race_category,gender_category,...,glyburide-metformin_category,glipizide-metformin_category,glimepiride-pioglitazone_category,metformin-rosiglitazone_category,metformin-pioglitazone_category,age_mean,age_category,change_category,diabetes-medication_category,diabetes
0,2,49,0,19,0,0,1,9,1,0,...,0,0,0,0,0,55,1,1,1,0
1,2,19,0,15,0,0,2,8,1,0,...,0,0,0,0,0,85,1,0,0,1
2,1,52,1,12,0,0,0,9,0,0,...,0,0,0,0,0,65,1,0,1,0
3,7,47,1,14,0,0,1,6,1,1,...,0,0,0,0,0,65,1,0,1,1
4,7,41,2,32,0,0,0,9,1,0,...,0,0,0,0,0,85,1,1,1,0


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73450 entries, 0 to 73449
Data columns (total 38 columns):
 #   Column                             Non-Null Count  Dtype
---  ------                             --------------  -----
 0   time_in_hospital                   73450 non-null  int64
 1   num_lab_procedures                 73450 non-null  int64
 2   num_procedures                     73450 non-null  int64
 3   num_medications                    73450 non-null  int64
 4   number_outpatient                  73450 non-null  int64
 5   number_emergency                   73450 non-null  int64
 6   number_inpatient                   73450 non-null  int64
 7   number_diagnoses                   73450 non-null  int64
 8   race_category                      73450 non-null  int64
 9   gender_category                    73450 non-null  int64
 10  metformin_category                 73450 non-null  int64
 11  repaglinide_category               73450 non-null  int64
 12  nateglinide_catego

## Mitigation utils


In [12]:
def train_test_splitting(df, n_splits):
  df_splitting = {}

  w = int(len(df)/n_splits)
  window = w
  start_point = 0
  for i in range(0,n_splits):
      train = {}
      test = {}
      df_train_1 = {}
      df_train_2 = {}
      df_test = df[start_point:window]
      if i != 0:
        df_train_1 = df[0: start_point]

      if i != n_splits-1:
        df_train_2 = df[window: len(df)]

      if (i != 0 and  i != n_splits-1):
        concat_df = [df_train_1, df_train_2]
        df_train = pd.concat(concat_df)
      elif i != 0:
        df_train = df_train_1
      else:
        df_train = df_train_2

      start_point= window
      window = window + w

      df_splitting[i] = {'train': df_train, 'test': df_test}
  return df_splitting

In [13]:
#print(train_test_splitting(df, n_splits))

In [14]:
def df_X_Y_split(df_train, df_test, target_variable):
  Y_train = df_train[target_variable]
  X_train = df_train.drop(target_variable, axis=1)
  Y_test = df_test[target_variable]
  X_test = df_test.drop(target_variable, axis=1)
  return X_train, Y_train, X_test, Y_test

In [15]:
def df_X_Y_split_2D(df_train, df_test, target_variable, sensible_attribute):
  Y_train = df_train[target_variable].values
  S_train = df_train[sensible_attribute].values
  X_train = df_train.drop(target_variable, axis=1)
  X_train = X_train.drop(sensible_attribute, axis=1)

  Y_test = df_test[target_variable]
  X_test = df_test.drop(target_variable, axis=1)
  S_test = df_test[sensible_attribute].values

  X_train = normalize(X_train)
  X_test = normalize(X_test)

  pca = PCA(n_components=2)
  X_train2D = pca.fit(X_train).transform(X_train)
  X_test2D = pca.fit(X_test).transform(X_test)
  return X_train2D, Y_train, X_test2D, Y_test, S_train, S_test

In [16]:
def df_X_Y_split_S(df_train, df_test, target_variable, sensible_attribute):
  Y_train = df_train[target_variable].values
  S_train = df_train[sensible_attribute].values
  X_train = df_train.drop(target_variable, axis=1)

  Y_test = df_test[target_variable]
  X_test = df_test.drop(target_variable, axis=1)
  S_test = df_test[sensible_attribute].values

  return X_train, Y_train, X_test, Y_test, S_train, S_test

In [17]:
def compute_predictions_and_tests(df, target_variable, n_splits, models, n_estimators, random_seed):
  predicted_and_real_values = {}
  for model_name in tqdm(models):
    clf = models[model_name]
    df_splitting = train_test_splitting(df, n_splits)
    pred_and_y = {}
    for i in range(0,n_splits):
      df_split = df_splitting[i]
      df_train = df_split['train']
      df_test = df_split['test']

      X_train, Y_train, X_test, Y_test = df_X_Y_split(df_train, df_test, target_variable)
      clf.fit(X_train,Y_train)
      y_pred = clf.predict(X_test)

      S_test = X_test[sensible_attribute].values
      pred_and_y[i] = {'y_test': Y_test.to_numpy().astype(int), 'y_pred': y_pred.astype(int), 's_test':  S_test.astype(int)}

    predicted_and_real_values[model_name] = pred_and_y

  return predicted_and_real_values


In [18]:
def fl_am_compute_predictions_and_tests(df, target_variable, sensible_attribute, n_splits, classifier):
  predicted_and_real_values = {}

  df_splitting = train_test_splitting(df, n_splits)
  pred_and_y = {}
  for i in range(0,n_splits):

    df_split = df_splitting[i]
    df_train = df_split['train']
    df_test = df_split['test']
    ##Input data should be in 2-dimensions
    X_train2D, Y_train, X_test2D, Y_test, S_train, S_test = df_X_Y_split_2D(df_train, df_test, target_variable, sensible_attribute)
    #print(len(X_train2D), len(Y_train),len(S_train))

    #classifier is given in in-processing mitigation
    predictor = classifier.fit(X_train2D, Y_train, sensitive_features=S_train)
    y_pred = predictor.predict(X_test2D)
    pred_and_y[i] = {'y_test': Y_test.to_numpy(), 'y_pred': y_pred, 's_test':  S_test }

  predicted_and_real_values = pred_and_y

  return predicted_and_real_values

In [19]:
def fl_to_compute_predictions_and_tests(df, target_variable, sensible_attribute, n_splits, classifier):
  predicted_and_real_values = {}

  df_splitting = train_test_splitting(df, n_splits)
  pred_and_y = {}
  for i in range(0,n_splits):

    df_split = df_splitting[i]
    df_train = df_split['train']
    df_test = df_split['test']
    ##Input data should be in 2-dimensions
    X_train, Y_train, X_test, Y_test, S_train, S_test = df_X_Y_split_S(df_train, df_test, target_variable, sensible_attribute)
    #print(len(X_train2D), len(Y_train),len(S_train))

    #classifier is given in in-processing mitigation
    predictor = classifier.fit(X_train, Y_train, sensitive_features=S_train)
    y_pred = predictor.predict(X_test, sensitive_features=S_test)
    pred_and_y[i] = {'y_test': Y_test.to_numpy(), 'y_pred': y_pred, 's_test':  S_test}

  predicted_and_real_values = pred_and_y

  return predicted_and_real_values

In [20]:
#print(compute_predictions(df, target_variable, n_splits, models, n_estimators, random_seed))

In [24]:
def save_predictions_and_tests(predicted_and_test, dataset_name, sensible_attribute, mitigation,path_to_project):
  save_path = path_to_project + '/data/predictions_and_tests/pred_test-{}-{}-{}.p'.format(dataset_name, sensible_attribute, mitigation)
  with open(save_path, 'wb') as fp:
    pickle.dump(predicted_and_test, fp, protocol=pickle.HIGHEST_PROTOCOL)

In [25]:
def save_mitigated_dataset(mitigated_dataset, path_to_project,dataset_name, sensible_attribute, mitigation):
  mitigated_dataset.to_csv("{}/data/mitigated/mitigated-{}-{}-{}.csv".format(path_to_project,dataset_name, sensible_attribute, mitigation), sep=',', index=False, encoding='utf-8')

In [23]:
def check_results(results):
  for model_name in tqdm(models):
    for i in range(0,n_splits):
      y_test = results[model_name][i]['y_test']
      y_pred = results[model_name][i]['y_pred']
      for j in range(0,67):
        if y_test[j] != y_pred[j]:
          print(y_test[j], y_pred[j])

# Fairlearn
[Fairlearn](https://fairlearn.org) is an open source toolkit that empowers developers of artificial intelligence (AI) systems to assess their system's fairness and mitigate any observed unfairness issues.

### Correlation Remover
Pre-processing: mitigated dataset changes

MITIGATED DATASET

ALPHA PARAMETER SET TO DEFAULT

NO MODEL ENFORCEMENT

In [26]:
def fl_cr(config):
  mitigation='fl-cr'
  df, target_variable,sensible_attribute, path_to_project, n_splits, models, n_estimators, random_seed = unpack_config(config)
  X_raw = df.drop(target_variable, axis=1)
  X_raw = pd.get_dummies(X_raw)
  y = df[target_variable].values

  cr = CorrelationRemover(sensitive_feature_ids=[sensible_attribute])
  X_cr = cr.fit_transform(X_raw)

  X_cr = pd.DataFrame(
      X_cr, columns = X_raw.drop(sensible_attribute, axis=1).columns
  )

  X_cr[sensible_attribute] = X_raw[sensible_attribute]
  mit_fl_cr = X_cr.copy(deep=True)
  mit_fl_cr[target_variable] = y

  save_mitigated_dataset(mit_fl_cr,path_to_project,dataset_name, sensible_attribute, mitigation)
  predictions_and_tests = compute_predictions_and_tests(mit_fl_cr, target_variable, n_splits, models, n_estimators, random_seed)
  save_predictions_and_tests(predictions_and_tests, mitigation, dataset_name, sensible_attribute, path_to_project)
  return mit_fl_cr, predictions_and_tests

In [27]:
print(fl_cr(config))

  0%|          | 0/6 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/usr/local/lib/python3.1

(
           time_in_hospital  num_lab_procedures  num_procedures  num_medications  \
0              2.021027           49.328073       -0.036802        18.756052   
1              2.021027           19.328073       -0.036802        14.756052   
2              0.934578           50.979236        1.114506        12.759017   
3              7.021027           47.328073        0.963198        13.756052   
4              7.021027           41.328073        1.963198        31.756052   
...                 ...                 ...             ...              ...   
73445          3.021027           27.328073        1.963198        17.756052   
73446          8.021027           76.328073        4.963198        33.756052   
73447          2.021027           35.328073        1.963198        14.756052   
73448          1.021027            1.328073        0.963198        16.756052   
73449          6.021027           40.328073       -0.036802        12.756052   

       number_outpatient  number_emergency  number_inpatient  \
0                    0.0               0.0          1.004734   
1                    0.0               0.0          2.004734   
2                    0.0               0.0         -0.014728   
3                    0.0               0.0          1.004734   
4                    0.0               0.0          0.004734   
...                  ...               ...               ...   
73445                0.0               0.0          6.004734   
73446                0.0               0.0          0.004734   
73447                0.0               0.0          1.004734   
73448                0.0               0.0          0.004734   
73449                0.0               0.0          1.004734   

       number_diagnoses  gender_category  metformin_category  ...  \
0              8.889340        -0.017107            0.998922  ...   
1              7.889340        -0.017107           -0.001078  ...   
2              9.344308         0.053227            0.003355  ...   
3              5.889340         0.982893           -0.001078  ...   
4              8.889340        -0.017107            0.998922  ...   
...                 ...              ...                 ...  ...   
73445          6.889340         0.982893           -0.001078  ...   
73446          8.889340         0.982893           -0.001078  ...   
73447          6.889340        -0.017107           -0.001078  ...   
73448          4.889340        -0.017107           -0.001078  ...   
73449          6.889340         0.982893            0.998922  ...   

       glipizide-metformin_category  glimepiride-pioglitazone_category  \
0                         -0.000026                          -0.000004   
1                         -0.000026                          -0.000004   
2                          0.000082                           0.000014   
3                         -0.000026                          -0.000004   
4                         -0.000026                          -0.000004   
...                             ...                                ...   
73445                     -0.000026                          -0.000004   
73446                     -0.000026                          -0.000004   
73447                     -0.000026                          -0.000004   
73448                     -0.000026                          -0.000004   
73449                     -0.000026                          -0.000004   

       metformin-rosiglitazone_category  metformin-pioglitazone_category  \
0                                   0.0                        -0.000004   
1                                   0.0                        -0.000004   
2                                   0.0                         0.000014   
3                                   0.0                        -0.000004   
4                                   0.0                        -0.000004   
...                                 ...                              ...   
73445         

### Threshold Optimizer
Post-processing: change predictions, mitigated dataset is equal to the original one

SET THE MODEL TO CLASSIFIER - NO MODEL ENFORCEMENT

NO PARAMETERS

NO MITIGATED DATASET

In [28]:
def fl_to(config):
  mitigation='fl-to'
  df, target_variable, sensible_attribute, path_to_project, n_splits, models, n_estimators, random_seed = unpack_config(config)
  predictions_and_tests= {}

  for model_name in tqdm(models):
    classifier =  models[model_name]
    TO = ThresholdOptimizer(estimator=classifier)
    pred_and_y = fl_to_compute_predictions_and_tests(df, target_variable, sensible_attribute, n_splits, TO)
    predictions_and_tests[model_name] = pred_and_y

  save_predictions_and_tests(predictions_and_tests, mitigation, dataset_name, sensible_attribute, path_to_project)
  return predictions_and_tests

In [29]:
print(fl_to(config))

  0%|          | 0/6 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/fairlearn/postprocessing/_threshold_optimizer.py:285: FutureWarning: 'predict_method' default value is changed from 'predict' to 'auto'. Explicitly pass `predict_method='predict' to replicate the old behavior, or pass `predict_method='auto' or other valid values to silence this warning.
  warn(
/usr/local/lib/python3.10/dist-packages/fairlearn/postprocessing/_threshold_optimizer.py:285: FutureWarning: 'predict_method' default value is changed from 'predict' to 'auto'. Explicitly pass `predict_method='predict' to replicate the old behavior, or pass `predict_method='auto' or other valid values to silence this warning.
  warn(
/usr/local/lib/python3.10/dist-packages/fairlearn/postprocessing/_threshold_optimizer.py:285: FutureWarning: 'predict_method' default value is changed from 'predict' to 'auto'. Explicitly pass `predict_method='predict' to replicate the old behavior, or pass `predict_method='auto' or other valid values to silence this warning.


{
    'Logistic Regression': {
        0: {
            'y_test': array([0, 1, 0, ..., 1, 0, 1]),
            'y_pred': array([1, 1, 0, ..., 0, 0, 1]),
            's_test': array([1, 1, 0, ..., 0, 1, 1])
        },
        1: {
            'y_test': array([1, 1, 0, ..., 1, 0, 0]),
            'y_pred': array([0, 0, 0, ..., 0, 0, 0]),
            's_test': array([1, 1, 0, ..., 1, 1, 1])
        },
        2: {
            'y_test': array([0, 1, 1, ..., 0, 0, 0]),
            'y_pred': array([1, 0, 0, ..., 0, 0, 0]),
            's_test': array([1, 1, 1, ..., 1, 1, 0])
        },
        3: {
            'y_test': array([1, 1, 0, ..., 0, 0, 1]),
            'y_pred': array([0, 0, 0, ..., 0, 0, 0]),
            's_test': array([1, 1, 0, ..., 1, 1, 1])
        },
        4: {
            'y_test': array([0, 1, 0, ..., 1, 1, 0]),
            'y_pred': array([0, 0, 0, ..., 0, 0, 0]),
            's_test': array([1, 1, 1, ..., 1, 1, 1])
        },
        5: {
            'y_test': array([0, 0, 1, ..., 1, 0, 1]),
            'y_pred': array([0, 0, 0, ..., 0, 0, 0]),
            's_test': array([1, 1, 1, ..., 1, 0, 0])
        },
        6: {
            'y_test': array([0, 1, 0, ..., 0, 0, 0]),
            'y_pred': array([0, 1, 0, ..., 0, 0, 0]),
            's_test': array([1, 1, 0, ..., 1, 0, 0])
        },
        7: {
            'y_test': array([1, 0, 0, ..., 0, 1, 1]),
            'y_pred': array([0, 0, 0, ..., 1, 1, 0]),
            's_test': array([0, 1, 1, ..., 1, 1, 1])
        },
        8: {
            'y_test': array([1, 0, 0, ..., 0, 0, 1]),
            'y_pred': array([1, 0, 0, ..., 0, 0, 0]),
            's_test': array([0, 1, 1, ..., 1, 1, 1])
        },
        9: {
            'y_test': array([0, 0, 0, ..., 0, 0, 0]),
            'y_pred': array([0, 0, 0, ..., 0, 0, 0]),
            's_test': array([1, 1, 1, ..., 1, 1, 1])
        }
    },
    'Decision Tree': {
        0: {
            'y_test': array([0, 1, 0, ..., 1, 0, 1]),
            'y_pred': array([1, 0, 1, ..., 1, 0, 1]),
            's_test': array([1, 1, 0, ..., 0, 1, 1])
        },
        1: {
            'y_test': array([1, 1, 0, ..., 1, 0, 0]),
            'y_pred': array([1, 1, 1, ..., 1, 1, 0]),
            's_test': array([1, 1, 0, ..., 1, 1, 1])
        },
        2: {
            'y_test': array([0, 1, 1, ..., 0, 0, 0]),
            'y_pred': array([0, 1, 0, ..., 0, 0, 1]),
            's_test': array([1, 1, 1, ..., 1, 1, 0])
        },
        3: {
            'y_test': array([1, 1, 0, ..., 0, 0, 1]),
            'y_pred': array([0, 1, 0, ..., 1, 0, 1]),
            's_test': array([1, 1, 0, ..., 1, 1, 1])
        },
        4: {
            'y_test': array([0, 1, 0, ..., 1, 1, 0]),
            'y_pred': array([0, 1, 0, ..., 1, 1, 0]),
            's_test': array([1, 1, 1, ..., 1, 1, 1])
        },
        5: {
            'y_test': array([0, 0, 1, ..., 1, 0, 1]),
            'y_pred': array([0, 0, 1, ..., 1, 1, 1]),
            's_test': array([1, 1, 1, ..., 1, 0, 0])
        },
        6: {
            'y_test': array([0, 1, 0, ..., 0, 0, 0]),
            'y_pred': array([1, 1, 1, ..., 0, 0, 1]),
            's_test': array([1, 1, 0, ..., 1, 0, 0])
        },
        7: {
            'y_test': array([1, 0, 0, ..., 0, 1, 1]),
            'y_pred': array([0, 0, 1, ..., 1, 1, 0]),
            's_test': array([0, 1, 1, ..., 1, 1, 1])
        },
        8: {
            'y_test': array([1, 0, 0, ..., 0, 0, 1]),
            'y_pred': array([1, 0, 1, ..., 0, 0, 0]),
            's_test': array([0, 1, 1, ..., 1, 1, 1])
        },
        9: {
            'y_test': array([0, 0, 0, ..., 0, 0, 0]),
            'y_pred': array([1, 0, 0, ..., 1, 0, 1]),
            's_test': array([1, 1, 1, ..., 1, 1, 1])
        }
    },
    'Bagging': {
        0: {
            'y_test': array([0, 1, 0, ..., 1, 0, 1]),
            'y_pred': array([1, 1, 0, ..., 0, 0, 1]),
            's_test': array([1, 1, 0, ..., 0, 1, 1])
        },
        1: {
            'y_test

# AIF360
[AIF360](https://aif360.mybluemix.net) is a system used in order to detect and mitigate a possible bias inside a dataset.
It requires firstly the identification of the corresponding protected attribute (sex, race, age etc.)

### Configuration

In order to use [AIF360](https://aif360.mybluemix.net) we need to define a Binary Dataset and set the privileged/unprivileged groups

In [30]:
data_orig_aif = BinaryLabelDataset(
    favorable_label = 1,
    unfavorable_label = 0,
    df = df.copy(),
    label_names = [target_variable],
    protected_attribute_names = [sensible_attribute])

privileged_groups = [{'race_category': 1}]  #todo mettere sensible_attribute
unprivileged_groups = [{'race_category': 0}]

print(data_orig_aif.privileged_protected_attributes)

[array([1.])]

## Pre-processing

### Reweighing

Pre-processing: dataset is equal to the original one, the predictor is the same, only use different weights

NO PARAMETER

NO MITIGATED DATASET

NO MODEL ENFORCEMENT

In [31]:
def aif360_rw(config):
  mitigation = 'aif360-rw'
  df, target_variable,sensible_attribute, path_to_project, n_splits, models, n_estimators, random_seed = unpack_config(config)

  predictions_and_tests = {}
  for model_name in models:
    clf = models[model_name]
    RW = Reweighing(unprivileged_groups = unprivileged_groups,
                privileged_groups = privileged_groups)

    df_splitting = train_test_splitting(df, n_splits)
    pred_and_y = {}
    for i in range(0,n_splits):
      df_split = df_splitting[i]
      df_train = df_split['train']
      df_test = df_split['test']

      X_train, Y_train, X_test, Y_test = df_X_Y_split(df_train, df_test, target_variable)
      S_test = df_test[sensible_attribute].values

      data_orig_aif = BinaryLabelDataset(favorable_label = 1, unfavorable_label = 0, df = df_train.copy(), label_names = [target_variable], protected_attribute_names = [sensible_attribute])
      rw_dataset = RW.fit_transform(data_orig_aif)
      rw_df = rw_dataset.convert_to_dataframe()
      sample_weights= rw_df[1]["instance_weights"]


      clf.fit(X_train,Y_train,sample_weight=sample_weights)
      y_pred = clf.predict(X_test)
      pred_and_y[i] = {'y_test': Y_test.to_numpy().astype(int), 'y_pred': y_pred.astype(int), 's_test':  S_test.astype(int)}

      predictions_and_tests[model_name] = pred_and_y

  save_predictions_and_tests(predictions_and_tests, mitigation, dataset_name, sensible_attribute, path_to_project)

  return predictions_and_tests

In [32]:
print(aif360_rw(config))

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/usr/local/lib/python3.1

{
    'Logistic Regression': {
        0: {
            'y_test': array([0, 1, 0, ..., 1, 0, 1]),
            'y_pred': array([1, 1, 0, ..., 0, 0, 1]),
            's_test': array([1, 1, 0, ..., 0, 1, 1])
        },
        1: {
            'y_test': array([1, 1, 0, ..., 1, 0, 0]),
            'y_pred': array([0, 0, 1, ..., 0, 0, 0]),
            's_test': array([1, 1, 0, ..., 1, 1, 1])
        },
        2: {
            'y_test': array([0, 1, 1, ..., 0, 0, 0]),
            'y_pred': array([1, 0, 0, ..., 0, 0, 0]),
            's_test': array([1, 1, 1, ..., 1, 1, 0])
        },
        3: {
            'y_test': array([1, 1, 0, ..., 0, 0, 1]),
            'y_pred': array([0, 0, 0, ..., 0, 0, 0]),
            's_test': array([1, 1, 0, ..., 1, 1, 1])
        },
        4: {
            'y_test': array([0, 1, 0, ..., 1, 1, 0]),
            'y_pred': array([0, 0, 0, ..., 0, 0, 0]),
            's_test': array([1, 1, 1, ..., 1, 1, 1])
        },
        5: {
            'y_test': array([0, 0, 1, ..., 1, 0, 1]),
            'y_pred': array([0, 0, 0, ..., 0, 0, 0]),
            's_test': array([1, 1, 1, ..., 1, 0, 0])
        },
        6: {
            'y_test': array([0, 1, 0, ..., 0, 0, 0]),
            'y_pred': array([0, 1, 0, ..., 0, 0, 0]),
            's_test': array([1, 1, 0, ..., 1, 0, 0])
        },
        7: {
            'y_test': array([1, 0, 0, ..., 0, 1, 1]),
            'y_pred': array([0, 0, 0, ..., 1, 1, 0]),
            's_test': array([0, 1, 1, ..., 1, 1, 1])
        },
        8: {
            'y_test': array([1, 0, 0, ..., 0, 0, 1]),
            'y_pred': array([1, 0, 0, ..., 0, 0, 0]),
            's_test': array([0, 1, 1, ..., 1, 1, 1])
        },
        9: {
            'y_test': array([0, 0, 0, ..., 0, 0, 0]),
            'y_pred': array([0, 0, 0, ..., 0, 0, 0]),
            's_test': array([1, 1, 1, ..., 1, 1, 1])
        }
    },
    'Decision Tree': {
        0: {
            'y_test': array([0, 1, 0, ..., 1, 0, 1]),
            'y_pred': array([0, 0, 1, ..., 0, 0, 1]),
            's_test': array([1, 1, 0, ..., 0, 1, 1])
        },
        1: {
            'y_test': array([1, 1, 0, ..., 1, 0, 0]),
            'y_pred': array([1, 1, 0, ..., 1, 0, 0]),
            's_test': array([1, 1, 0, ..., 1, 1, 1])
        },
        2: {
            'y_test': array([0, 1, 1, ..., 0, 0, 0]),
            'y_pred': array([0, 0, 0, ..., 0, 1, 1]),
            's_test': array([1, 1, 1, ..., 1, 1, 0])
        },
        3: {
            'y_test': array([1, 1, 0, ..., 0, 0, 1]),
            'y_pred': array([0, 1, 0, ..., 0, 0, 1]),
            's_test': array([1, 1, 0, ..., 1, 1, 1])
        },
        4: {
            'y_test': array([0, 1, 0, ..., 1, 1, 0]),
            'y_pred': array([0, 1, 0, ..., 1, 1, 0]),
            's_test': array([1, 1, 1, ..., 1, 1, 1])
        },
        5: {
            'y_test': array([0, 0, 1, ..., 1, 0, 1]),
            'y_pred': array([0, 0, 1, ..., 1, 0, 1]),
            's_test': array([1, 1, 1, ..., 1, 0, 0])
        },
        6: {
            'y_test': array([0, 1, 0, ..., 0, 0, 0]),
            'y_pred': array([1, 1, 0, ..., 0, 0, 1]),
            's_test': array([1, 1, 0, ..., 1, 0, 0])
        },
        7: {
            'y_test': array([1, 0, 0, ..., 0, 1, 1]),
            'y_pred': array([0, 0, 1, ..., 1, 1, 0]),
            's_test': array([0, 1, 1, ..., 1, 1, 1])
        },
        8: {
            'y_test': array([1, 0, 0, ..., 0, 0, 1]),
            'y_pred': array([1, 0, 1, ..., 0, 0, 0]),
            's_test': array([0, 1, 1, ..., 1, 1, 1])
        },
        9: {
            'y_test': array([0, 0, 0, ..., 0, 0, 0]),
            'y_pred': array([1, 0, 0, ..., 0, 0, 1]),
            's_test': array([1, 1, 1, ..., 1, 1, 1])
        }
    },
    'Bagging': {
        0: {
            'y_test': array([0, 1, 0, ..., 1, 0, 1]),
            'y_pred': array([1, 1, 0, ..., 0, 0, 1]),
            's_test': array([1, 1, 0, ..., 0, 1, 1])
        },
        1: {
            'y_test

### Disparate Impact Remover

Pre-processing: It transforms the dataset in a new mitigated one

PARAMETER REPAIR LEVEL

MITIGATED DATASET

NO MODEL ENFORCEMENT

In [34]:
def aif360_di(config):
  mitigation = 'aif360-di'
  df, target_variable,sensible_attribute, path_to_project, n_splits, models, n_estimators, random_seed = unpack_config(config)

  predictions_and_tests = {}

  #levels = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
  DIR = DisparateImpactRemover(repair_level=0.5)
  data_orig_aif = BinaryLabelDataset(favorable_label = 1, unfavorable_label = 0, df = df.copy(), label_names = [target_variable], protected_attribute_names = [sensible_attribute])

  train_repd = DIR.fit_transform(data_orig_aif)
  mit_aif360_di = train_repd.convert_to_dataframe()[0]

  save_mitigated_dataset(mit_aif360_di,path_to_project,dataset_name, sensible_attribute, mitigation)

  predictions_and_tests = compute_predictions_and_tests(mit_aif360_di, target_variable, n_splits, models, n_estimators, random_seed)
  save_predictions_and_tests(predictions_and_tests, mitigation, dataset_name, sensible_attribute, path_to_project)

  return predictions_and_tests

In [35]:
print(aif360_di(config))

  0%|          | 0/6 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/usr/local/lib/python3.1

{
    'Logistic Regression': {
        0: {
            'y_test': array([0, 1, 0, ..., 1, 0, 1]),
            'y_pred': array([1, 1, 0, ..., 0, 0, 1]),
            's_test': array([1, 1, 0, ..., 0, 1, 1])
        },
        1: {
            'y_test': array([1, 1, 0, ..., 1, 0, 0]),
            'y_pred': array([0, 0, 0, ..., 0, 0, 0]),
            's_test': array([1, 1, 0, ..., 1, 1, 1])
        },
        2: {
            'y_test': array([0, 1, 1, ..., 0, 0, 0]),
            'y_pred': array([1, 0, 0, ..., 0, 0, 0]),
            's_test': array([1, 1, 1, ..., 1, 1, 0])
        },
        3: {
            'y_test': array([1, 1, 0, ..., 0, 0, 1]),
            'y_pred': array([0, 0, 0, ..., 0, 0, 0]),
            's_test': array([1, 1, 0, ..., 1, 1, 1])
        },
        4: {
            'y_test': array([0, 1, 0, ..., 1, 1, 0]),
            'y_pred': array([0, 0, 0, ..., 0, 0, 0]),
            's_test': array([1, 1, 1, ..., 1, 1, 1])
        },
        5: {
            'y_test': array([0, 0, 1, ..., 1, 0, 1]),
            'y_pred': array([0, 0, 0, ..., 0, 0, 0]),
            's_test': array([1, 1, 1, ..., 1, 0, 0])
        },
        6: {
            'y_test': array([0, 1, 0, ..., 0, 0, 0]),
            'y_pred': array([0, 1, 0, ..., 0, 0, 0]),
            's_test': array([1, 1, 0, ..., 1, 0, 0])
        },
        7: {
            'y_test': array([1, 0, 0, ..., 0, 1, 1]),
            'y_pred': array([0, 0, 0, ..., 1, 1, 0]),
            's_test': array([0, 1, 1, ..., 1, 1, 1])
        },
        8: {
            'y_test': array([1, 0, 0, ..., 0, 0, 1]),
            'y_pred': array([1, 0, 0, ..., 0, 0, 0]),
            's_test': array([0, 1, 1, ..., 1, 1, 1])
        },
        9: {
            'y_test': array([0, 0, 0, ..., 0, 0, 0]),
            'y_pred': array([0, 0, 0, ..., 0, 0, 0]),
            's_test': array([1, 1, 1, ..., 1, 1, 1])
        }
    },
    'Decision Tree': {
        0: {
            'y_test': array([0, 1, 0, ..., 1, 0, 1]),
            'y_pred': array([0, 0, 1, ..., 1, 0, 1]),
            's_test': array([1, 1, 0, ..., 0, 1, 1])
        },
        1: {
            'y_test': array([1, 1, 0, ..., 1, 0, 0]),
            'y_pred': array([1, 1, 1, ..., 1, 0, 0]),
            's_test': array([1, 1, 0, ..., 1, 1, 1])
        },
        2: {
            'y_test': array([0, 1, 1, ..., 0, 0, 0]),
            'y_pred': array([0, 1, 1, ..., 0, 0, 1]),
            's_test': array([1, 1, 1, ..., 1, 1, 0])
        },
        3: {
            'y_test': array([1, 1, 0, ..., 0, 0, 1]),
            'y_pred': array([0, 1, 0, ..., 0, 0, 1]),
            's_test': array([1, 1, 0, ..., 1, 1, 1])
        },
        4: {
            'y_test': array([0, 1, 0, ..., 1, 1, 0]),
            'y_pred': array([0, 1, 0, ..., 1, 1, 0]),
            's_test': array([1, 1, 1, ..., 1, 1, 1])
        },
        5: {
            'y_test': array([0, 0, 1, ..., 1, 0, 1]),
            'y_pred': array([0, 0, 1, ..., 1, 0, 1]),
            's_test': array([1, 1, 1, ..., 1, 0, 0])
        },
        6: {
            'y_test': array([0, 1, 0, ..., 0, 0, 0]),
            'y_pred': array([1, 1, 0, ..., 0, 0, 1]),
            's_test': array([1, 1, 0, ..., 1, 0, 0])
        },
        7: {
            'y_test': array([1, 0, 0, ..., 0, 1, 1]),
            'y_pred': array([0, 0, 1, ..., 1, 1, 0]),
            's_test': array([0, 1, 1, ..., 1, 1, 1])
        },
        8: {
            'y_test': array([1, 0, 0, ..., 0, 0, 1]),
            'y_pred': array([1, 0, 1, ..., 1, 0, 0]),
            's_test': array([0, 1, 1, ..., 1, 1, 1])
        },
        9: {
            'y_test': array([0, 0, 0, ..., 0, 0, 0]),
            'y_pred': array([1, 0, 0, ..., 0, 0, 1]),
            's_test': array([1, 1, 1, ..., 1, 1, 1])
        }
    },
    'Bagging': {
        0: {
            'y_test': array([0, 1, 0, ..., 1, 0, 1]),
            'y_pred': array([1, 1, 0, ..., 0, 0, 1]),
            's_test': array([1, 1, 0, ..., 0, 1, 1])
        },
        1: {
            'y_test

### Learning Fair Representation

PARAMETERS k, Ax, Ay, Az maxitem e maxfun

MITIGATED DATASET

NO MODEL ENFORCEMENT

In [36]:
def aif360_lfr(config):
  mitigation = 'aif360-lfr'
  df, target_variable,sensible_attribute, path_to_project, n_splits, models, n_estimators, random_seed = unpack_config(config)

  predictions_and_tests = {}

  TR = LFR(unprivileged_groups=unprivileged_groups,
    privileged_groups=privileged_groups,
    seed= random_seed, k=10, Ax=0.01, Ay=1.0, Az=50.0,
    verbose=1)
  data_orig_aif = BinaryLabelDataset(favorable_label = 1, unfavorable_label = 0, df = df.copy(), label_names = [target_variable], protected_attribute_names = [sensible_attribute])

  TR = TR.fit(data_orig_aif, maxiter=500, maxfun=500) # Changed 5000 to 500
  transf_dataset = TR.transform(data_orig_aif)
  mit_aif360_lfr = transf_dataset.convert_to_dataframe()[0]

  save_mitigated_dataset(mit_aif360_lfr, path_to_project, dataset_name, sensible_attribute, mitigation)

  predictions_and_tests = compute_predictions_and_tests(mit_aif360_lfr, target_variable, n_splits, models, n_estimators, random_seed)
  save_predictions_and_tests(predictions_and_tests, mitigation, dataset_name, sensible_attribute, path_to_project)

  return predictions_and_tests

In [37]:
results= aif360_lfr(config)
print(results)

step: 0, loss: 4.489957762216563, L_x: 369.2447499538556,  L_y: 0.746639884569254,  L_z: 0.0010174075621750545
step: 250, loss: 4.489957760965653, L_x: 369.24475005866645,  L_y: 0.7466398878527873,  L_z: 0.001017407450524016
step: 500, loss: 4.435140042180883, L_x: 369.15332674720935,  L_y: 0.6997766437296784,  L_z: 0.0008766026195822438
step: 750, loss: 4.435140046291497, L_x: 369.1533269141086,  L_y: 0.6997766422380226,  L_z: 0.0008766026982477679


  0%|          | 0/6 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/usr/local/lib/python3.1

{
    'Logistic Regression': {
        0: {
            'y_test': array([1, 1, 1, ..., 1, 1, 1]),
            'y_pred': array([1, 1, 1, ..., 1, 1, 1]),
            's_test': array([1, 1, 0, ..., 0, 1, 1])
        },
        1: {
            'y_test': array([1, 1, 1, ..., 1, 1, 1]),
            'y_pred': array([1, 1, 1, ..., 1, 1, 1]),
            's_test': array([1, 1, 0, ..., 1, 1, 1])
        },
        2: {
            'y_test': array([1, 1, 1, ..., 1, 1, 1]),
            'y_pred': array([1, 1, 1, ..., 1, 1, 1]),
            's_test': array([1, 1, 1, ..., 1, 1, 0])
        },
        3: {
            'y_test': array([1, 1, 1, ..., 1, 1, 1]),
            'y_pred': array([1, 1, 1, ..., 1, 1, 1]),
            's_test': array([1, 1, 0, ..., 1, 1, 1])
        },
        4: {
            'y_test': array([1, 1, 1, ..., 1, 1, 1]),
            'y_pred': array([1, 1, 1, ..., 1, 1, 1]),
            's_test': array([1, 1, 1, ..., 1, 1, 1])
        },
        5: {
            'y_test': array([1, 1, 1, ..., 1, 1, 1]),
            'y_pred': array([1, 1, 1, ..., 1, 1, 1]),
            's_test': array([1, 1, 1, ..., 1, 0, 0])
        },
        6: {
            'y_test': array([1, 1, 1, ..., 1, 1, 1]),
            'y_pred': array([1, 1, 1, ..., 1, 1, 1]),
            's_test': array([1, 1, 0, ..., 1, 0, 0])
        },
        7: {
            'y_test': array([1, 1, 1, ..., 1, 1, 1]),
            'y_pred': array([1, 1, 1, ..., 1, 1, 1]),
            's_test': array([0, 1, 1, ..., 1, 1, 1])
        },
        8: {
            'y_test': array([1, 1, 1, ..., 1, 1, 1]),
            'y_pred': array([1, 1, 1, ..., 1, 1, 1]),
            's_test': array([0, 1, 1, ..., 1, 1, 1])
        },
        9: {
            'y_test': array([1, 1, 1, ..., 1, 1, 1]),
            'y_pred': array([1, 1, 1, ..., 1, 1, 1]),
            's_test': array([1, 1, 1, ..., 1, 1, 1])
        }
    },
    'Decision Tree': {
        0: {
            'y_test': array([1, 1, 1, ..., 1, 1, 1]),
            'y_pred': array([1, 1, 1, ..., 1, 1, 1]),
            's_test': array([1, 1, 0, ..., 0, 1, 1])
        },
        1: {
            'y_test': array([1, 1, 1, ..., 1, 1, 1]),
            'y_pred': array([1, 1, 1, ..., 1, 1, 1]),
            's_test': array([1, 1, 0, ..., 1, 1, 1])
        },
        2: {
            'y_test': array([1, 1, 1, ..., 1, 1, 1]),
            'y_pred': array([1, 1, 1, ..., 1, 1, 1]),
            's_test': array([1, 1, 1, ..., 1, 1, 0])
        },
        3: {
            'y_test': array([1, 1, 1, ..., 1, 1, 1]),
            'y_pred': array([1, 1, 1, ..., 1, 1, 1]),
            's_test': array([1, 1, 0, ..., 1, 1, 1])
        },
        4: {
            'y_test': array([1, 1, 1, ..., 1, 1, 1]),
            'y_pred': array([1, 1, 1, ..., 1, 1, 1]),
            's_test': array([1, 1, 1, ..., 1, 1, 1])
        },
        5: {
            'y_test': array([1, 1, 1, ..., 1, 1, 1]),
            'y_pred': array([1, 1, 1, ..., 1, 1, 1]),
            's_test': array([1, 1, 1, ..., 1, 0, 0])
        },
        6: {
            'y_test': array([1, 1, 1, ..., 1, 1, 1]),
            'y_pred': array([1, 1, 1, ..., 1, 1, 1]),
            's_test': array([1, 1, 0, ..., 1, 0, 0])
        },
        7: {
            'y_test': array([1, 1, 1, ..., 1, 1, 1]),
            'y_pred': array([1, 1, 1, ..., 1, 1, 1]),
            's_test': array([0, 1, 1, ..., 1, 1, 1])
        },
        8: {
            'y_test': array([1, 1, 1, ..., 1, 1, 1]),
            'y_pred': array([1, 1, 1, ..., 1, 1, 1]),
            's_test': array([0, 1, 1, ..., 1, 1, 1])
        },
        9: {
            'y_test': array([1, 1, 1, ..., 1, 1, 1]),
            'y_pred': array([1, 1, 1, ..., 1, 1, 1]),
            's_test': array([1, 1, 1, ..., 1, 1, 1])
        }
    },
    'Bagging': {
        0: {
            'y_test': array([1, 1, 1, ..., 1, 1, 1]),
            'y_pred': array([1, 1, 1, ..., 1, 1, 1]),
            's_test': array([1, 1, 0, ..., 0, 1, 1])
        },
        1: {
            'y_test

### Optimized Preprocessing

Pre-processing: mitigated dataset
Richiede valori binari come input quindi è stato fatto un pre-processing sui valori per essere 0,1

OptTool difficile da creare, chiesto a Daniel delucidazioni

PARAMETERS distortion_fun, epsilon, clist, dlist

MITIGATED DATASET

NO MODEL ENFORCEMENT

In [38]:
def get_distortion_diabetes(vold,vnew): # NEED TO CHANGE THIS
        bad_val=3
        OutNew=vnew["diabetes"]
        OutOld=vold["diabetes"]

        if not apply_dimensionality_reduction:
          InsOld=vold["num_lab_procedures"]
          InsNew=vnew["num_lab_procedures"]
        else:
          InsOld = 0
          InsNew = 0

        if ((OutNew>OutOld)& (InsNew<InsOld)): #| ((OutNew>OutOld) & (GluNew<GluOld)):
            return bad_val
        else:
            return 0

In [39]:
def distortion_function(df):
  df2 = df.copy()
  df2['time_in_hospital'] = np.where(df2['time_in_hospital'].between(1,6), 1, 0) # 1 valore giusto, 0 anomalo
  df2["number_inpatient"]=np.where(df2["number_inpatient"] < 1, 1, 0)
  df2["number_diagnoses"]=np.where(df2["number_diagnoses"].between(1,10), 1, 0)

  if not apply_dimensionality_reduction:
    df2 = df2.drop(columns=['glipizide_category', 'glyburide_category']) # drop non-binary columns that are full of zero's
    df2["num_procedures"]=np.where(df2["num_procedures"] < 5, 1, 0)
    df2["num_medications"]=np.where(df2["num_medications"] < 20, 1, 0)
    df2['num_lab_procedures'] = np.where(df2['num_lab_procedures'] < 50, 1, 0) # based on distribution

  return df2

In [40]:
def aif360_op(config):
  mitigation = 'aif360-op'
  df, target_variable, sensible_attribute, path_to_project, n_splits, models, n_estimators, random_seed = unpack_config(config)

  if apply_dimensionality_reduction:
    df_op = distortion_function(df_reduced)
  else:
    df_op = distortion_function(df)

  data_orig_aif_op = BinaryLabelDataset(
    favorable_label = 1,
    unfavorable_label = 0,
    df = df_op.copy(),
    label_names = [target_variable],
    protected_attribute_names = [sensible_attribute],
    metadata = default_mappings)

  optim_options = {
    "distortion_fun": get_distortion_diabetes,
    "epsilon": 0.1,
    "clist": [0.99, 1.99, 2.99],
    "dlist": [.1, 0.05, 0]
  }

  predictions_and_tests = {}
  OP = OptimPreproc(OptTools, optim_options)
  dataset_orig_train, dataset_orig_test = data_orig_aif_op.split([0.7], shuffle=True)

  OP_fitted= OP.fit(dataset_orig_train)
  transf_dataset = OP_fitted.transform(data_orig_aif_op, transform_Y=True)
  mit_aif360_op = transf_dataset.convert_to_dataframe()[0]

  save_mitigated_dataset(mit_aif360_op, path_to_project, dataset_name, sensible_attribute, mitigation)

  predictions_and_tests = compute_predictions_and_tests(mit_aif360_op, target_variable, n_splits, models, n_estimators, random_seed)
  save_predictions_and_tests(predictions_and_tests, mitigation, dataset_name, sensible_attribute, path_to_project)

  return predictions_and_tests

In [41]:
print(aif360_op(config))

Output hidden; open in https://colab.research.google.com to view.

##In-processing

Ogni tecnica genera un nuovo classificatore: no score and no mitigated dataset

### Adversarial Debiasing UGUALE A FAIRLEARN (ORIGINALE)
In-processing: dataset is equal to the original one, the predictor is fairer by construction and given

PARAMETER SESSION

NO MITIGATED DATASET

NO SCORE FUNCTION

MODEL ENFORCEMENT

In [42]:
def aif360_ad(config):
  mitigation = 'aif360-ad'
  df, target_variable,sensible_attribute, path_to_project, n_splits, models, n_estimators, random_seed = unpack_config(config)

  predictions_and_tests = {}

  df_splitting = train_test_splitting(df, n_splits)
  pred_and_y = {}
  for i in range(0,n_splits):
    # Create a new session for each fold
    sess = tf.Session()
    # Added a unique scope name using the fold index
    AD = AdversarialDebiasing(privileged_groups = privileged_groups,
                        unprivileged_groups = unprivileged_groups,
                        scope_name=f'plain_classifier_{i}', # Changed scope name
                        debias=False,
                        sess=sess)

    df_split = df_splitting[i]
    df_train = df_split['train']
    df_test = df_split['test']

    X_train, Y_train, X_test, Y_test, S_train, S_test = df_X_Y_split_S(df_train, df_test, target_variable, sensible_attribute)

    data_orig_aif_train = BinaryLabelDataset(favorable_label = 1,
      unfavorable_label = 0,
      df = df_train.copy(),
      label_names = [target_variable],
      protected_attribute_names = [sensible_attribute])

    data_orig_aif_test = BinaryLabelDataset(favorable_label = 1,
      unfavorable_label = 0,
      df = df_test.copy(),
      label_names = [target_variable],
      protected_attribute_names = [sensible_attribute])

    AD.fit(data_orig_aif_train)

    predictions = AD.predict(data_orig_aif_test)

    mit_aif360_ad = predictions.convert_to_dataframe()[0]
    y_pred = mit_aif360_ad[target_variable]
    pred_and_y[i] = {'y_test': Y_test.to_numpy(), 'y_pred': y_pred.to_numpy().astype(int), 's_test':  S_test}

    predicted_and_real_tests = pred_and_y

    # Close the session after each fold
    sess.close()
  print(predicted_and_real_tests)
  save_predictions_and_tests(predicted_and_real_tests, mitigation, dataset_name, sensible_attribute, path_to_project)
  return predicted_and_real_tests

In [43]:
aif360_ad(config)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


epoch 0; iter: 0; batch classifier loss: 3.436237
epoch 0; iter: 200; batch classifier loss: 0.970226
epoch 0; iter: 400; batch classifier loss: 0.758126
epoch 1; iter: 0; batch classifier loss: 0.670420
epoch 1; iter: 200; batch classifier loss: 0.711980
epoch 1; iter: 400; batch classifier loss: 0.616865
epoch 2; iter: 0; batch classifier loss: 0.687254
epoch 2; iter: 200; batch classifier loss: 0.689853
epoch 2; iter: 400; batch classifier loss: 0.636264
epoch 3; iter: 0; batch classifier loss: 0.663160
epoch 3; iter: 200; batch classifier loss: 0.719068
epoch 3; iter: 400; batch classifier loss: 0.669326
epoch 4; iter: 0; batch classifier loss: 0.649026
epoch 4; iter: 200; batch classifier loss: 0.696810
epoch 4; iter: 400; batch classifier loss: 0.642341
epoch 5; iter: 0; batch classifier loss: 0.677828
epoch 5; iter: 200; batch classifier loss: 0.651003
epoch 5; iter: 400; batch classifier loss: 0.655933
epoch 6; iter: 0; batch classifier loss: 0.658612
epoch 6; iter: 200; batch 

{
    0: {
        'y_test': array([0, 1, 0, ..., 1, 0, 1]),
        'y_pred': array([1, 1, 0, ..., 0, 0, 1]),
        's_test': array([1, 1, 0, ..., 0, 1, 1])
    },
    1: {
        'y_test': array([1, 1, 0, ..., 1, 0, 0]),
        'y_pred': array([1, 0, 1, ..., 0, 0, 0]),
        's_test': array([1, 1, 0, ..., 1, 1, 1])
    },
    2: {
        'y_test': array([0, 1, 1, ..., 0, 0, 0]),
        'y_pred': array([1, 0, 0, ..., 0, 0, 0]),
        's_test': array([1, 1, 1, ..., 1, 1, 0])
    },
    3: {
        'y_test': array([1, 1, 0, ..., 0, 0, 1]),
        'y_pred': array([0, 1, 1, ..., 0, 0, 0]),
        's_test': array([1, 1, 0, ..., 1, 1, 1])
    },
    4: {
        'y_test': array([0, 1, 0, ..., 1, 1, 0]),
        'y_pred': array([0, 0, 0, ..., 0, 0, 0]),
        's_test': array([1, 1, 1, ..., 1, 1, 1])
    },
    5: {
        'y_test': array([0, 0, 1, ..., 1, 0, 1]),
        'y_pred': array([0, 0, 0, ..., 0, 0, 0]),
        's_test': array([1, 1, 1, ..., 1, 0, 0])
    },
    6: {
        'y_test': array([0, 1, 0, ..., 0, 0, 0]),
        'y_pred': array([0, 1, 0, ..., 0, 0, 0]),
        's_test': array([1, 1, 0, ..., 1, 0, 0])
    },
    7: {
        'y_test': array([1, 0, 0, ..., 0, 1, 1]),
        'y_pred': array([0, 0, 0, ..., 1, 1, 0]),
        's_test': array([0, 1, 1, ..., 1, 1, 1])
    },
    8: {
        'y_test': array([1, 0, 0, ..., 0, 0, 1]),
        'y_pred': array([1, 1, 0, ..., 0, 0, 0]),
        's_test': array([0, 1, 1, ..., 1, 1, 1])
    },
    9: {
        'y_test': array([0, 0, 0, ..., 0, 0, 0]),
        'y_pred': array([0, 0, 0, ..., 1, 0, 0]),
        's_test': array([1, 1, 1, ..., 1, 1, 1])
    }
}

{0: {'y_test': array([0, 1, 0, ..., 1, 0, 1]),
  'y_pred': array([1, 1, 0, ..., 0, 0, 1]),
  's_test': array([1, 1, 0, ..., 0, 1, 1])},
 1: {'y_test': array([1, 1, 0, ..., 1, 0, 0]),
  'y_pred': array([1, 0, 1, ..., 0, 0, 0]),
  's_test': array([1, 1, 0, ..., 1, 1, 1])},
 2: {'y_test': array([0, 1, 1, ..., 0, 0, 0]),
  'y_pred': array([1, 0, 0, ..., 0, 0, 0]),
  's_test': array([1, 1, 1, ..., 1, 1, 0])},
 3: {'y_test': array([1, 1, 0, ..., 0, 0, 1]),
  'y_pred': array([0, 1, 1, ..., 0, 0, 0]),
  's_test': array([1, 1, 0, ..., 1, 1, 1])},
 4: {'y_test': array([0, 1, 0, ..., 1, 1, 0]),
  'y_pred': array([0, 0, 0, ..., 0, 0, 0]),
  's_test': array([1, 1, 1, ..., 1, 1, 1])},
 5: {'y_test': array([0, 0, 1, ..., 1, 0, 1]),
  'y_pred': array([0, 0, 0, ..., 0, 0, 0]),
  's_test': array([1, 1, 1, ..., 1, 0, 0])},
 6: {'y_test': array([0, 1, 0, ..., 0, 0, 0]),
  'y_pred': array([0, 1, 0, ..., 0, 0, 0]),
  's_test': array([1, 1, 0, ..., 1, 0, 0])},
 7: {'y_test': array([1, 0, 0, ..., 0, 1, 1]),
 

### Prejudice remover
In-processing: dataset is equal to the original one, the predictor is fairer by construction and given

NON FUNZIONA!

PARAMETER eta

NO MITIGATED DATASET

NO SCORE FUNCTION

MODEL ENFORCEMENT



In [44]:
def aif360_pr(config):
  mitigation = 'aif360-pr'
  df, target_variable,sensible_attribute, path_to_project, n_splits, models, n_estimators, random_seed = unpack_config(config)

  predictions_and_tests = {}

  df_splitting = train_test_splitting(df, n_splits)
  pred_and_y = {}
  for i in range(0,n_splits):

    PR = PrejudiceRemover(eta=1.0, sensitive_attr=sensible_attribute, class_attr=target_variable)

    df_split = df_splitting[i]
    df_train = df_split['train']
    df_test = df_split['test']

    X_train, Y_train, X_test, Y_test, S_train, S_test = df_X_Y_split_S(df_train, df_test, target_variable, sensible_attribute)

    data_orig_aif_train = BinaryLabelDataset(favorable_label = 1,
      unfavorable_label = 0,
      df = df_train.copy(),
      label_names = [target_variable],
      protected_attribute_names = [sensible_attribute])

    data_orig_aif_test = BinaryLabelDataset(favorable_label = 1,
      unfavorable_label = 0,
      df = df_test.copy(),
      label_names = [target_variable],
      protected_attribute_names = [sensible_attribute])

    PR.fit(data_orig_aif_train)
    predictions = PR.predict(data_orig_aif_test)
    mit_aif360_pr = predictions.convert_to_dataframe()[0]

    y_pred = mit_aif360_pr[target_variable]
    pred_and_y[i] = {'y_test': Y_test.to_numpy(), 'y_pred': y_pred.to_numpy().astype(int), 's_test':  S_test}

    predicted_and_real_tests = pred_and_y
  print(predicted_and_real_tests)

  save_predictions_and_tests(predicted_and_real_tests, mitigation, dataset_name, sensible_attribute, path_to_project)
  return predicted_and_real_tests



In [45]:
aif360_pr(config)

{
    0: {
        'y_test': array([0, 1, 0, ..., 1, 0, 1]),
        'y_pred': array([1, 1, 0, ..., 0, 0, 1]),
        's_test': array([1, 1, 0, ..., 0, 1, 1])
    },
    1: {
        'y_test': array([1, 1, 0, ..., 1, 0, 0]),
        'y_pred': array([0, 0, 1, ..., 0, 0, 0]),
        's_test': array([1, 1, 0, ..., 1, 1, 1])
    },
    2: {
        'y_test': array([0, 1, 1, ..., 0, 0, 0]),
        'y_pred': array([1, 0, 0, ..., 0, 0, 0]),
        's_test': array([1, 1, 1, ..., 1, 1, 0])
    },
    3: {
        'y_test': array([1, 1, 0, ..., 0, 0, 1]),
        'y_pred': array([0, 0, 0, ..., 0, 0, 0]),
        's_test': array([1, 1, 0, ..., 1, 1, 1])
    },
    4: {
        'y_test': array([0, 1, 0, ..., 1, 1, 0]),
        'y_pred': array([0, 0, 0, ..., 0, 0, 0]),
        's_test': array([1, 1, 1, ..., 1, 1, 1])
    },
    5: {
        'y_test': array([0, 0, 1, ..., 1, 0, 1]),
        'y_pred': array([0, 0, 0, ..., 0, 0, 0]),
        's_test': array([1, 1, 1, ..., 1, 0, 0])
    },
    6: {
        'y_test': array([0, 1, 0, ..., 0, 0, 0]),
        'y_pred': array([0, 1, 0, ..., 0, 0, 0]),
        's_test': array([1, 1, 0, ..., 1, 0, 0])
    },
    7: {
        'y_test': array([1, 0, 0, ..., 0, 1, 1]),
        'y_pred': array([0, 0, 0, ..., 1, 1, 0]),
        's_test': array([0, 1, 1, ..., 1, 1, 1])
    },
    8: {
        'y_test': array([1, 0, 0, ..., 0, 0, 1]),
        'y_pred': array([1, 0, 0, ..., 0, 0, 0]),
        's_test': array([0, 1, 1, ..., 1, 1, 1])
    },
    9: {
        'y_test': array([0, 0, 0, ..., 0, 0, 0]),
        'y_pred': array([0, 0, 0, ..., 0, 0, 0]),
        's_test': array([1, 1, 1, ..., 1, 1, 1])
    }
}

{0: {'y_test': array([0, 1, 0, ..., 1, 0, 1]),
  'y_pred': array([1, 1, 0, ..., 0, 0, 1]),
  's_test': array([1, 1, 0, ..., 0, 1, 1])},
 1: {'y_test': array([1, 1, 0, ..., 1, 0, 0]),
  'y_pred': array([0, 0, 1, ..., 0, 0, 0]),
  's_test': array([1, 1, 0, ..., 1, 1, 1])},
 2: {'y_test': array([0, 1, 1, ..., 0, 0, 0]),
  'y_pred': array([1, 0, 0, ..., 0, 0, 0]),
  's_test': array([1, 1, 1, ..., 1, 1, 0])},
 3: {'y_test': array([1, 1, 0, ..., 0, 0, 1]),
  'y_pred': array([0, 0, 0, ..., 0, 0, 0]),
  's_test': array([1, 1, 0, ..., 1, 1, 1])},
 4: {'y_test': array([0, 1, 0, ..., 1, 1, 0]),
  'y_pred': array([0, 0, 0, ..., 0, 0, 0]),
  's_test': array([1, 1, 1, ..., 1, 1, 1])},
 5: {'y_test': array([0, 0, 1, ..., 1, 0, 1]),
  'y_pred': array([0, 0, 0, ..., 0, 0, 0]),
  's_test': array([1, 1, 1, ..., 1, 0, 0])},
 6: {'y_test': array([0, 1, 0, ..., 0, 0, 0]),
  'y_pred': array([0, 1, 0, ..., 0, 0, 0]),
  's_test': array([1, 1, 0, ..., 1, 0, 0])},
 7: {'y_test': array([1, 0, 0, ..., 0, 1, 1]),
 

### Exponentiated Gradient Reduction
In-processing: dataset is equal to the original one, the predictor is fairer by construction and given

PARAMETERS constraint e estimator

NO MITIGATED DATASET

NO SCORE FUNCTION

MODEL ENFORCEMENT

In [46]:
def aif360_er(config):
  mitigation = 'aif360-er'
  df, target_variable,sensible_attribute, path_to_project, n_splits, models, n_estimators, random_seed = unpack_config(config)
  df_splitting = train_test_splitting(df, n_splits)
  pred_and_y = {}
  predictions_and_tests = {}
  for i in range(0,n_splits):

    df_split = df_splitting[i]
    df_train = df_split['train']
    df_test = df_split['test']

    X_train, Y_train, X_test, Y_test, S_train, S_test = df_X_Y_split_S(df_train, df_test, target_variable, sensible_attribute)

    data_orig_aif_train = BinaryLabelDataset(favorable_label = 1,
      unfavorable_label = 0,
      df = df_train.copy(),
      label_names = [target_variable],
      protected_attribute_names = [sensible_attribute])

    data_orig_aif_test = BinaryLabelDataset(favorable_label = 1,
      unfavorable_label = 0,
      df = df_test.copy(),
      label_names = [target_variable],
      protected_attribute_names = [sensible_attribute])

    lmod = DecisionTreeClassifier(max_depth=50)
    #lmod = LogisticRegression(solver='newton-cholesky', max_iter=500) # Not converging, is this normal?? #liblinear, sag, newton-cholesky

    exp_grad_red = ExponentiatedGradientReduction(estimator=lmod, constraints='DemographicParity', drop_prot_attr=False)
    exp_grad_red.fit(data_orig_aif_train)
    predictions = exp_grad_red.predict(data_orig_aif_test)

    mit_aif360_er = predictions.convert_to_dataframe()[0]
    y_pred = mit_aif360_er[target_variable]
    pred_and_y[i] = {'y_test': Y_test.to_numpy(), 'y_pred': y_pred.to_numpy().astype(int), 's_test':  S_test}

  predictions_and_tests = pred_and_y

  print(predictions_and_tests)
  save_predictions_and_tests(predictions_and_tests, mitigation, dataset_name, sensible_attribute, path_to_project)
  return predictions_and_tests


In [47]:
aif360_er(config)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/fairlearn/reductions/_moments/utility_parity.py:185: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updatin

{
    0: {
        'y_test': array([0, 1, 0, ..., 1, 0, 1]),
        'y_pred': array([0, 0, 1, ..., 0, 1, 1]),
        's_test': array([1, 1, 0, ..., 0, 1, 1])
    },
    1: {
        'y_test': array([1, 1, 0, ..., 1, 0, 0]),
        'y_pred': array([1, 1, 0, ..., 1, 0, 0]),
        's_test': array([1, 1, 0, ..., 1, 1, 1])
    },
    2: {
        'y_test': array([0, 1, 1, ..., 0, 0, 0]),
        'y_pred': array([0, 0, 0, ..., 0, 0, 1]),
        's_test': array([1, 1, 1, ..., 1, 1, 0])
    },
    3: {
        'y_test': array([1, 1, 0, ..., 0, 0, 1]),
        'y_pred': array([0, 0, 0, ..., 1, 0, 1]),
        's_test': array([1, 1, 0, ..., 1, 1, 1])
    },
    4: {
        'y_test': array([0, 1, 0, ..., 1, 1, 0]),
        'y_pred': array([0, 1, 0, ..., 1, 1, 0]),
        's_test': array([1, 1, 1, ..., 1, 1, 1])
    },
    5: {
        'y_test': array([0, 0, 1, ..., 1, 0, 1]),
        'y_pred': array([0, 1, 1, ..., 1, 1, 1]),
        's_test': array([1, 1, 1, ..., 1, 0, 0])
    },
    6: {
        'y_test': array([0, 1, 0, ..., 0, 0, 0]),
        'y_pred': array([1, 1, 0, ..., 0, 0, 1]),
        's_test': array([1, 1, 0, ..., 1, 0, 0])
    },
    7: {
        'y_test': array([1, 0, 0, ..., 0, 1, 1]),
        'y_pred': array([1, 0, 1, ..., 1, 1, 0]),
        's_test': array([0, 1, 1, ..., 1, 1, 1])
    },
    8: {
        'y_test': array([1, 0, 0, ..., 0, 0, 1]),
        'y_pred': array([1, 0, 1, ..., 0, 0, 0]),
        's_test': array([0, 1, 1, ..., 1, 1, 1])
    },
    9: {
        'y_test': array([0, 0, 0, ..., 0, 0, 0]),
        'y_pred': array([1, 0, 0, ..., 0, 1, 1]),
        's_test': array([1, 1, 1, ..., 1, 1, 1])
    }
}

{0: {'y_test': array([0, 1, 0, ..., 1, 0, 1]),
  'y_pred': array([0, 0, 1, ..., 0, 1, 1]),
  's_test': array([1, 1, 0, ..., 0, 1, 1])},
 1: {'y_test': array([1, 1, 0, ..., 1, 0, 0]),
  'y_pred': array([1, 1, 0, ..., 1, 0, 0]),
  's_test': array([1, 1, 0, ..., 1, 1, 1])},
 2: {'y_test': array([0, 1, 1, ..., 0, 0, 0]),
  'y_pred': array([0, 0, 0, ..., 0, 0, 1]),
  's_test': array([1, 1, 1, ..., 1, 1, 0])},
 3: {'y_test': array([1, 1, 0, ..., 0, 0, 1]),
  'y_pred': array([0, 0, 0, ..., 1, 0, 1]),
  's_test': array([1, 1, 0, ..., 1, 1, 1])},
 4: {'y_test': array([0, 1, 0, ..., 1, 1, 0]),
  'y_pred': array([0, 1, 0, ..., 1, 1, 0]),
  's_test': array([1, 1, 1, ..., 1, 1, 1])},
 5: {'y_test': array([0, 0, 1, ..., 1, 0, 1]),
  'y_pred': array([0, 1, 1, ..., 1, 1, 1]),
  's_test': array([1, 1, 1, ..., 1, 0, 0])},
 6: {'y_test': array([0, 1, 0, ..., 0, 0, 0]),
  'y_pred': array([1, 1, 0, ..., 0, 0, 1]),
  's_test': array([1, 1, 0, ..., 1, 0, 0])},
 7: {'y_test': array([1, 0, 0, ..., 0, 1, 1]),
 

## Post-processing

In the first phase there is the model building: the same for Calibrated Equalized Post-processing, Equalize Odds Post_processing and Reject Option Classification


In [48]:
def compute_predictions(data_orig_aif, model):
  # Get the dataset and split into train and validation set
  dataset_orig_train, data_orig_valid = data_orig_aif.split([0.5], shuffle=True, seed=random_seed)

  dataset_orig_train_pred = dataset_orig_train.copy(deepcopy=True)
  data_orig_valid_pred = data_orig_valid.copy(deepcopy=True)

  X_train = dataset_orig_train.features
  y_train = dataset_orig_train.labels.ravel()

  model.fit(X_train, y_train)
  y_train_pred = model.predict(X_train)

  # positive class index
  pos_ind = np.where(model.classes_ == dataset_orig_train.favorable_label)[0][0]

  dataset_orig_train_pred.labels = y_train_pred

  X_valid = data_orig_valid_pred.features
  y_valid = data_orig_valid_pred.labels
  data_orig_valid_pred.scores = model.predict_proba(X_valid)[:,pos_ind].reshape(-1,1)

  return data_orig_valid, data_orig_valid_pred

### Calibrated Equalized Post-Processing

Post-processing: model's labels are changes, mitigated dataset is equal to the original one

PARAMETER cost_constraint

NO MITIGATED DATASET

NO EMBEDDED SCORE FUNCTION

NO MODEL ENFORCEMENT

In [49]:
def aif360_ce(config):
  mitigation = 'aif360-ce'
  df, target_variable,sensible_attribute, path_to_project, n_splits, models, n_estimators, random_seed = unpack_config(config)

  predictions_and_tests = {}
  for model_name in models:
    clf = models[model_name]
    CEPP = CalibratedEqOddsPostprocessing(unprivileged_groups= unprivileged_groups, privileged_groups = privileged_groups, cost_constraint='weighted', seed=random_seed)

    df_splitting = train_test_splitting(df, n_splits)
    pred_and_y = {}
    for i in range(0,n_splits):
      df_split = df_splitting[i]
      df_train = df_split['train']
      df_test = df_split['test']

      X_train, Y_train, X_test, Y_test = df_X_Y_split(df_train, df_test, target_variable)
      S_test = df_test[sensible_attribute].values

      data_orig_aif_train = BinaryLabelDataset(favorable_label = 1,
      unfavorable_label = 0,
      df = df_train.copy(),
      label_names = [target_variable],
      protected_attribute_names = [sensible_attribute])

      data_orig_aif_test = BinaryLabelDataset(favorable_label = 1,
      unfavorable_label = 0,
      df = df_test.copy(),
      label_names = [target_variable],
      protected_attribute_names = [sensible_attribute])

      data_orig_valid, data_orig_valid_pred = compute_predictions(data_orig_aif_train, clf)

      cpp = CEPP.fit(data_orig_valid, data_orig_valid_pred)
      predictions = cpp.predict(data_orig_aif_test)
      mit_aif360_cpp = predictions.convert_to_dataframe()[0]
      y_pred = mit_aif360_cpp[target_variable]
      pred_and_y[i] = {'y_test': Y_test.to_numpy(), 'y_pred': y_pred.to_numpy().astype(int), 's_test':  S_test}

      predictions_and_tests[model_name] = pred_and_y

  print(predictions_and_tests)

  save_predictions_and_tests(predictions_and_tests, mitigation, dataset_name, sensible_attribute, path_to_project)

  return predictions_and_tests

In [50]:
aif360_ce(config)

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/usr/local/lib/python3.1

{
    'Logistic Regression': {
        0: {
            'y_test': array([0, 1, 0, ..., 1, 0, 1]),
            'y_pred': array([0, 1, 0, ..., 1, 0, 1]),
            's_test': array([1, 1, 0, ..., 0, 1, 1])
        },
        1: {
            'y_test': array([1, 1, 0, ..., 1, 0, 0]),
            'y_pred': array([1, 1, 0, ..., 1, 0, 0]),
            's_test': array([1, 1, 0, ..., 1, 1, 1])
        },
        2: {
            'y_test': array([0, 1, 1, ..., 0, 0, 0]),
            'y_pred': array([0, 1, 1, ..., 0, 0, 0]),
            's_test': array([1, 1, 1, ..., 1, 1, 0])
        },
        3: {
            'y_test': array([1, 1, 0, ..., 0, 0, 1]),
            'y_pred': array([1, 1, 0, ..., 0, 0, 1]),
            's_test': array([1, 1, 0, ..., 1, 1, 1])
        },
        4: {
            'y_test': array([0, 1, 0, ..., 1, 1, 0]),
            'y_pred': array([0, 1, 0, ..., 1, 1, 0]),
            's_test': array([1, 1, 1, ..., 1, 1, 1])
        },
        5: {
            'y_test': array([0, 0, 1, ..., 1, 0, 1]),
            'y_pred': array([0, 0, 1, ..., 1, 0, 1]),
            's_test': array([1, 1, 1, ..., 1, 0, 0])
        },
        6: {
            'y_test': array([0, 1, 0, ..., 0, 0, 0]),
            'y_pred': array([0, 1, 0, ..., 0, 0, 0]),
            's_test': array([1, 1, 0, ..., 1, 0, 0])
        },
        7: {
            'y_test': array([1, 0, 0, ..., 0, 1, 1]),
            'y_pred': array([0, 0, 0, ..., 0, 1, 1]),
            's_test': array([0, 1, 1, ..., 1, 1, 1])
        },
        8: {
            'y_test': array([1, 0, 0, ..., 0, 0, 1]),
            'y_pred': array([0, 0, 0, ..., 0, 0, 1]),
            's_test': array([0, 1, 1, ..., 1, 1, 1])
        },
        9: {
            'y_test': array([0, 0, 0, ..., 0, 0, 0]),
            'y_pred': array([0, 0, 0, ..., 0, 0, 0]),
            's_test': array([1, 1, 1, ..., 1, 1, 1])
        }
    },
    'Decision Tree': {
        0: {
            'y_test': array([0, 1, 0, ..., 1, 0, 1]),
            'y_pred': array([0, 1, 0, ..., 1, 0, 1]),
            's_test': array([1, 1, 0, ..., 0, 1, 1])
        },
        1: {
            'y_test': array([1, 1, 0, ..., 1, 0, 0]),
            'y_pred': array([1, 1, 0, ..., 1, 0, 0]),
            's_test': array([1, 1, 0, ..., 1, 1, 1])
        },
        2: {
            'y_test': array([0, 1, 1, ..., 0, 0, 0]),
            'y_pred': array([0, 1, 1, ..., 0, 0, 0]),
            's_test': array([1, 1, 1, ..., 1, 1, 0])
        },
        3: {
            'y_test': array([1, 1, 0, ..., 0, 0, 1]),
            'y_pred': array([1, 1, 0, ..., 0, 0, 1]),
            's_test': array([1, 1, 0, ..., 1, 1, 1])
        },
        4: {
            'y_test': array([0, 1, 0, ..., 1, 1, 0]),
            'y_pred': array([0, 1, 0, ..., 1, 1, 0]),
            's_test': array([1, 1, 1, ..., 1, 1, 1])
        },
        5: {
            'y_test': array([0, 0, 1, ..., 1, 0, 1]),
            'y_pred': array([0, 0, 1, ..., 1, 0, 1]),
            's_test': array([1, 1, 1, ..., 1, 0, 0])
        },
        6: {
            'y_test': array([0, 1, 0, ..., 0, 0, 0]),
            'y_pred': array([0, 1, 0, ..., 0, 0, 0]),
            's_test': array([1, 1, 0, ..., 1, 0, 0])
        },
        7: {
            'y_test': array([1, 0, 0, ..., 0, 1, 1]),
            'y_pred': array([0, 0, 0, ..., 0, 1, 1]),
            's_test': array([0, 1, 1, ..., 1, 1, 1])
        },
        8: {
            'y_test': array([1, 0, 0, ..., 0, 0, 1]),
            'y_pred': array([1, 0, 0, ..., 0, 0, 1]),
            's_test': array([0, 1, 1, ..., 1, 1, 1])
        },
        9: {
            'y_test': array([0, 0, 0, ..., 0, 0, 0]),
            'y_pred': array([0, 0, 0, ..., 0, 0, 0]),
            's_test': array([1, 1, 1, ..., 1, 1, 1])
        }
    },
    'Bagging': {
        0: {
            'y_test': array([0, 1, 0, ..., 1, 0, 1]),
            'y_pred': array([0, 1, 0, ..., 1, 0, 1]),
            's_test': array([1, 1, 0, ..., 0, 1, 1])
        },
        1: {
            'y_test

{'Logistic Regression': {0: {'y_test': array([0, 1, 0, ..., 1, 0, 1]),
   'y_pred': array([0, 1, 0, ..., 1, 0, 1]),
   's_test': array([1, 1, 0, ..., 0, 1, 1])},
  1: {'y_test': array([1, 1, 0, ..., 1, 0, 0]),
   'y_pred': array([1, 1, 0, ..., 1, 0, 0]),
   's_test': array([1, 1, 0, ..., 1, 1, 1])},
  2: {'y_test': array([0, 1, 1, ..., 0, 0, 0]),
   'y_pred': array([0, 1, 1, ..., 0, 0, 0]),
   's_test': array([1, 1, 1, ..., 1, 1, 0])},
  3: {'y_test': array([1, 1, 0, ..., 0, 0, 1]),
   'y_pred': array([1, 1, 0, ..., 0, 0, 1]),
   's_test': array([1, 1, 0, ..., 1, 1, 1])},
  4: {'y_test': array([0, 1, 0, ..., 1, 1, 0]),
   'y_pred': array([0, 1, 0, ..., 1, 1, 0]),
   's_test': array([1, 1, 1, ..., 1, 1, 1])},
  5: {'y_test': array([0, 0, 1, ..., 1, 0, 1]),
   'y_pred': array([0, 0, 1, ..., 1, 0, 1]),
   's_test': array([1, 1, 1, ..., 1, 0, 0])},
  6: {'y_test': array([0, 1, 0, ..., 0, 0, 0]),
   'y_pred': array([0, 1, 0, ..., 0, 0, 0]),
   's_test': array([1, 1, 0, ..., 1, 0, 0])},
  7:

### Equalize Odds Post-Processing
Post-processing: model's labels are changes, mitigated dataset is equal to the original one

NON FUNZIONA!

NO PARAMETER

NO MITIGATED DATASET

NO EMBEDDED SCORE FUNCTION

NO MODEL ENFORCEMENT


IT DOES NOT WORK??

In [51]:
def aif360_eo(config):
  mitigation = 'aif360-eo'
  df, target_variable,sensible_attribute, path_to_project, n_splits, models, n_estimators, random_seed = unpack_config(config)

  predictions_and_tests = {}
  for model_name in models:
    clf = models[model_name]
    EOPP = EqOddsPostprocessing(unprivileged_groups= unprivileged_groups, privileged_groups = privileged_groups, seed=random_seed)

    df_splitting = train_test_splitting(df, n_splits)
    pred_and_y = {}
    for i in range(0,n_splits):
      df_split = df_splitting[i]
      df_train = df_split['train']
      df_test = df_split['test']

      X_train, Y_train, X_test, Y_test = df_X_Y_split(df_train, df_test, target_variable)
      S_test = df_test[sensible_attribute].values

      data_orig_aif_train = BinaryLabelDataset(favorable_label = 1,
      unfavorable_label = 0,
      df = df_train.copy(),
      label_names = [target_variable],
      protected_attribute_names = [sensible_attribute])

      data_orig_aif_test = BinaryLabelDataset(favorable_label = 1,
      unfavorable_label = 0,
      df = df_test.copy(),
      label_names = [target_variable],
      protected_attribute_names = [sensible_attribute])

      data_orig_valid, data_orig_valid_pred = compute_predictions(data_orig_aif_train, clf)

      eopp = EOPP.fit(data_orig_valid, data_orig_valid_pred)
      predictions = eopp.predict(data_orig_aif_test)
      mit_aif360_eopp = predictions.convert_to_dataframe()[0]
      y_pred = mit_aif360_eopp[target_variable]
      pred_and_y[i] = {'y_test': Y_test.to_numpy(), 'y_pred': y_pred.to_numpy().astype(int), 's_test':  S_test}

      predictions_and_tests[model_name] = pred_and_y

  print(predictions_and_tests)

  save_predictions_and_tests(predictions_and_tests, mitigation, dataset_name, sensible_attribute, path_to_project)

  return predictions_and_tests

In [ ]:
aif360_eo(config)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

KeyboardInterrupt: 

### Reject Option Classification

Post-processing: model's labels are changes, mitigated dataset is equal to the original one

NO MITIGATED DATASET

PARAMETERS: low_class_thresh,high_class_thresh, num_class_thresh, num_ROC_margin, metric_name, metric_ub, metric_lb

NO EMBEDDED SCORE FUCTION

NO MODEL ENFORCEMENT

In [1]:
def aif360_roc(config):
  mitigation = 'aif360-roc'
  df, target_variable,sensible_attribute, path_to_project, n_splits, models, n_estimators, random_seed = unpack_config(config)

  if apply_dimensionality_reduction:
    df = df_reduced

  print(df.shape)

  predictions_and_tests = {}
  for model_name in tqdm(models):
    clf = models[model_name]
    ROC = RejectOptionClassification(unprivileged_groups=unprivileged_groups,
                                     privileged_groups=privileged_groups,
                                     low_class_thresh=0.01, high_class_thresh=0.99,
                                     num_class_thresh=100, num_ROC_margin=50,
                                     metric_name="Equal opportunity difference", metric_ub=0.05, metric_lb=-0.05)

    df_splitting = train_test_splitting(df, n_splits)
    pred_and_y = {}
    for i in range(0,n_splits):
      df_split = df_splitting[i]
      df_train = df_split['train']
      df_test = df_split['test']

      X_train, Y_train, X_test, Y_test = df_X_Y_split(df_train, df_test, target_variable)
      S_test = df_test[sensible_attribute].values.astype(int)

      data_orig_aif_train = BinaryLabelDataset(favorable_label = 1,
      unfavorable_label = 0,
      df = df_train.copy(),
      label_names = [target_variable],
      protected_attribute_names = [sensible_attribute])

      data_orig_aif_test = BinaryLabelDataset(favorable_label = 1,
      unfavorable_label = 0,
      df = df_test.copy(),
      label_names = [target_variable],
      protected_attribute_names = [sensible_attribute])

      data_orig_valid, data_orig_valid_pred = compute_predictions(data_orig_aif_train, clf)

      roc = ROC.fit(data_orig_valid, data_orig_valid_pred)
      predictions = roc.predict(data_orig_aif_test)
      mit_aif360_roc = predictions.convert_to_dataframe()[0]
      y_pred = mit_aif360_roc[target_variable]
      pred_and_y[i] = {'y_test': Y_test.to_numpy(), 'y_pred': y_pred.to_numpy().astype(int), 's_test':  S_test}

      predictions_and_tests[model_name] = pred_and_y

  print(predictions_and_tests)

  save_predictions_and_tests(predictions_and_tests, mitigation, dataset_name, sensible_attribute, path_to_project)

  return predictions_and_tests

In [2]:
aif360_roc(config)

NameError: name 'config' is not defined